在本教程中，將學習如何使用**“管道模型(Pipeline)”**來清理您的建模代碼。


# 介紹

**管道模型(pipeline)**是保持數據預處理和建模程式碼井井有條的一種簡單方法。具體來說，管道模型將預處理和建模步驟固定起來，因此我們可以像使用單個捆綁包一樣使用整個捆綁包。

許多數據科學家無需pipeline即可一起破解模型，但是管道具有一些重要的好處。這些包括：

1. **更簡潔的程式碼：**在預處理的每個步驟中考慮數據可能會變得混亂。使用管道模型，我們無需在每個步驟中手動追蹤訓練數據和驗證數據。
2. **更少的錯誤：**使用步驟錯誤或忘記預處理步驟的機會更少。
3. **易於產品化(Productionize)：**很難將模型從原型過渡到可大規模部署的模型。在這裡我們不會涉及許多相關問題，但是管道可以提供幫助。
4. **更多模型驗證選項：**我們將在下一教程中看到一個示例，其中涉及交叉驗證。

# 舉例

與上一教程中一樣，我們將使用[墨爾本住房數據集]（https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home）。

我們不會專注於數據載入步驟。相反，我們假設自己到了在 `X_train` ， `X_valid` ， `y_train` 和 `y_valid` 中已經有訓練和驗證數據的步驟。

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('./data_set/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns 
                    if X_train_full[cname].nunique() < 10 
                    and X_train_full[cname].dtype == "object"
                   ]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

我們使用下面的 `head()` 方法來查看訓練數據。 請注意，數據包含分類數據和缺少值的列。 使用 pipeline，可以輕鬆處理這兩個問題！

In [6]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


我們分成三個步驟構建完整的 pipeline。

### 步驟1：定義預處理步驟

類似於 pipeline 如何將預處理步驟和建模步驟捆綁在一起，我們使用 `ColumnTransformer` 類(class)將不同的預處理步驟綁在一起。 以下是程式碼：

- 在 ** 數值型 ** 數據中插入缺失值，並且
- 對 ** 類別型 ** 數據使用 one-hot encoding 插補缺失值。

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])